# Simulation

In [48]:
import numpy as np

## Parameters

In [49]:
density_water = 1000  # kg/m^3
density_object = 500  # kg/m^3
object_volume = 0.1  # m^3
g = 9.81  # m/s^2
time_step = 0.01  # s
time_max = 10  # s

In [50]:
time = 0  # s
depth = 0  # m
velocity = 0  # m/s
acceleration = 0  # m/s^2
buoyancy =   density_water*object_volume*g
weight =   density_object*object_volume*g #n

In [51]:
# lists for plotting later
time = np.arange(0, time_max, time_step)
position = np.zeros_like(time)
velocity = np.zeros_like(time)
acceleration = np.zeros_like(time)

In [52]:
# simulation loop
for i in range(1, len(time)):
    # calculate the force on the object

    force = buoyancy - weight
    # calculate acceleration
    acceleration[i] = force / (weight/g)
    # calculate velocity
   # velocity[i-1] = acceleration[i-1]*time[i] #+ velocity[i-1]
    velocity[i] = acceleration[i-1]*time_step + velocity[i-1]
    # calculate position
    position[i] = velocity[i-1]*time_step + position[i-1]


# Plot the results

In [53]:
import matplotlib.pyplot as plt
import numpy as np
import math


In [12]:

#plt.plot(time, position, label="Position")
#plt.plot(time, velocity, label="Velocity")
#plt.plot(time, acceleration, label="Acceleration")
#plt.xlabel("Time (s)")
#plt.ylabel("Position (m), Velocity (m/s), Acceleration (m/s^2)")
#plt.legend()
#plt.show()
def calculate_acceleration(F, m): 
     
        a = F/m   #since f = ma, by dividing m from both sides, we yield acceleration
        """in this question, we calculate the acceleation of an object given the force appllied 
        to it and its mass, where F is he force applied in N and m is the mass in kg"""
        return a

def calculate_angular_acceleration(tau, I): 
      """calculates the torque applied to an object given the force 
      applied to it and the distance from the axis of rotation to the point where the force is applied."""
      aac = tau/I  
      return aac 

def calculate_torque(F_magnitude, F_direction, r):
     
       """calculates the torque applied to an object given the force applied to it and 
       the distance from the axis of rotation to the point where the force is applied."""
       F_dir = np.deg2rad(F_direction)
       torque = r*F_magnitude* np.cos(F_dir) + r*F_magnitude* np.sin(F_dir)
       return torque

def calculate_moment_of_inertia(m, r):
       """calculates the moment of inertia of an object given 
       its mass and the distance from the axis of rotation to 
       the center of mass of the object."""
       mi = m*r*r
       return mi

def calculate_auv_acceleration(F_magitude, F_angle, mass) :
       """calculates the acceleration of the AUV in the 2D plane."""
       #mass = 100 #kg
       #volume = 0.1 #m^3
       accx = F_magitude*np.cos(F_angle)/ mass
       accy = F_magitude*np.sin(F_angle)/ mass
       acc_total = accx + accy 
       return np.array([[accx, accy]]) #

def calculate_auv_angular_acceleration(F_magnitude, F_angle, inertia, thruster_distance):
       """calculates the angular acceleration of the AUV."""
       angular_acc = calculate_torque.torque(F_magnitude, F_angle, thruster_distance)*thruster_distance / inertia
       return angular_acc


def calculate_auv2_acceleration(T, alpha, mass, theta):
       """"Calculate the acceleration of an AUV given it has 4 thrusters"""
       """alpha is the system angle in degree, T is a 4x1 matrix, and mass in kg, theta is the global angle in degree"""
       component_matrix= np.array(
                    [ [np.cos(alpha), np.cos(alpha), -np.cos(alpha), -np.cos(alpha)],
                     [np.sin(alpha), -np.sin(alpha), -np.sin(alpha), np.sin(alpha)] ]
              ) 
       rotation_matrix= np.array(
                    [ [np.cos(theta), -np.sin(theta)],
                     [np.sin(theta), np.cos(theta)] ]
              ) 
       if T.shape == (4, 1):
              
              fxyprime = np.matmul(component_matrix, T) 
              
              acceleration_fxy = (np.matmul(rotation_matrix, fxyprime))/mass
              
             # acceleration_mag = math.sqrt(sum(pow(element, 2) for element in acceleration_fxy))
              return acceleration_fxy
       else: 
              raise TypeError("dimension invalid")

def calculate_auv2_angular_acceleration(T, L, l, alpha, inertia):
       """Calculate the angular acceleration of a 4 thruster AUV, aka the torque acceleration"""
       component_matrix= np.array(
                    [ [np.cos(alpha), np.cos(alpha), -np.cos(alpha), -np.cos(alpha)],
                     [np.sin(alpha), -np.sin(alpha), -np.sin(alpha), np.sin(alpha)] ]
              ) 
       force_fxy = np.matmul(component_matrix, T)
       accl = np.array([force_fxy[0] / inertia, force_fxy[1] / inertia])
       r = np.sqrt(L*L +l*l)
       mag = np.sqrt(force_fxy[0]**2 + force_fxy[1]**2)
       return calculate_torque(mag, alpha, r)

def simulation_auv2_motion(T, alpha, L, l, inertia=100, dt=.1, t_final=10, x0=0, y0=0, theta0=0):
       #time_step = 0.01
       # dt is essentially timestep
      # state = np.array([x0], [y0], [theta0])
       time = np.arange(0, t_final, dt)
       x = np.zeros_like(time)
       y = np.zeros_like(time)
       x[0]= x0
       y[0]=y0
       position = np.zeros_like(time)
       velocity = np.zeros_like(time) 
       angular_displacement = np.zeros_like(time)
       angular_acceleration = np.zeros_like(time)
       theta = np.zeros_like(time)
       theta[0] = theta0
       mass = 100 #kg
       omega= np.zeros_like(time) 
       a = np.tile(np.zeros_like(time), (2, 1))
       a[0] = calculate_auv2_acceleration(T, alpha, mass, theta)[0][0]
       omega[0] = calculate_auv2_angular_acceleration(T, L, l, alpha, inertia)[0][0]


      
       for i in range(1, len(time)):
              angular_acceleration[i] = calculate_auv2_angular_acceleration(T, L, l, alpha, inertia)
              a[0][i] = calculate_auv2_acceleration(T, alpha, mass, theta)[0][0]
              a[1][i] = calculate_auv2_acceleration(T, alpha, mass, theta)[0][1]
              velocity[i] = a[i-1]*dt + velocity[i-1]
              x[i] = velocity[i-1]*dt + x[i-1]
              y[i] = velocity[i-1]*dt + y[i-1]
              theta[i] = .5 * a[i]* dt**2 + velocity[i-1]*time[i-1]
              omega[i] = angular_acceleration[i-1]* dt + omega [i-1]
              time[i]= time[i] + time[i-1] 
             
       physicsdict = {
              "acceleration": a,
              "velocity": velocity, 
              "x": x,
              "y": y,
              "theta": theta,
              "omega": omega, 
              "time": time
       }
       return physicsdict


    #simulation_auv2_motion(np.array([[0], [50], [50], [30]]), np.pi/4, 1, 1, 1)
     


In [2]:
def plot_auv2(physicsdict):
        x = physicsdict["x"]
        y = physicsdict["y"]
        
        plt.plot(x, y)
        plt.xlabel("X Position")
        plt.ylabel("Y Position")
        plt.title("AUV Positions")
        plt.grid(True)
        plt.show()

In [29]:
def calculate_auv2_acceleration(T, alpha, mass, theta):
       """"Calculate the acceleration of an AUV given it has 4 thrusters"""
       """alpha is the system angle in degree, T is a 4x1 matrix, and mass in kg, theta is the global angle in degree"""
       if T.shape == (4, 1):
              component_matrix= np.array(
                    [ [np.cos(alpha), np.cos(alpha), -np.cos(alpha), -np.cos(alpha)],
                     [np.sin(alpha), -np.sin(alpha), -np.sin(alpha), np.sin(alpha)] ]
              ) 
              rotation_matrix= np.array(
                    [ [np.cos(theta), -np.sin(theta)],
                     [np.sin(theta), np.cos(theta)] ]
              ) 
              fxyprime = np.matmul(component_matrix, T) 
              
              acceleration_fxy = (np.matmul(rotation_matrix, fxyprime))/mass
              
             # acceleration_mag = math.sqrt(sum(pow(element, 2) for element in acceleration_fxy))
              return acceleration_fxy
       else: 
              raise TypeError("dimension invalid")
       
      

In [31]:
import numpy as np
#def simulation_auv2_motion(T, alpha, L, l, inertia=100, dt=.1, t_final=10, x0=0, y0=0, theta0=0):
physicsdict= simulation_auv2_motion(np.array([0], [50], [50], [30]), np.pi/3, 1, 1, .1, 10, 0, 0, 0)
plot_auv2(physicsdict) 



TypeError: array() takes from 1 to 2 positional arguments but 4 were given